# Interactive Review Process Example

Authors: Costa, Dellunde, and Rivas-Barragan, 2022

Example XX of the article submitted to Fuzzy sets and systems by Costa and Dellunde (December, 2022).
Note that this program allows modifying the example by changing the evaluation of the reviewers, the general chair constraint, the method for evaluating the submitted papers or the number of papers submitted.

### How to run the code
lorem ipsum

#### Preconditions:

## Example

### decisions.txt
```
reject
borderline
regular
accepted
```

### reviews.txt
```csv
reviewer,paper,decision
1, 1, regular
1, 2, reject-borderline
2, 1, reject-borderline-regular
2, 2, regular-accepted
3, 3, accepted
3, 4, borderline
4, 3, regular-accepted
4, 4, borderline
```

In [63]:
#@title 0. Generate example
decisions_txt = f"""\
reject
borderline
regular
accepted\
"""
with open("decisions.txt", 'w') as f:
  f.write(decisions_txt)

reviews_txt = f"""\
reviewer,paper,decision
1, 1, regular
1, 2, reject-borderline
2, 1, reject-borderline-regular
2, 2, regular-accepted
3, 3, accepted
3, 4, borderline
4, 3, regular-accepted
4, 4, borderline\
"""

with open("reviews.txt", 'w') as f:
  f.write(reviews_txt)

In [65]:
#@title 1. Read decisions' list and reviews
from tabulate import tabulate

def read_decisions(): # -> dict[str, float]:
  with open("decisions.txt") as f:
    decisions = f.read().splitlines()

  decision2score = {
      decision: round(i/(len(decisions)-1), 3) for i, decision in enumerate(decisions)
  }
  return decision2score

decision2score = read_decisions()
N = [score for score in decision2score.values()]

print(tabulate({
    "Decision": [d for d in decision2score.keys()],
    "Score": [s for s in decision2score.values()],
}, headers="keys"))

Decision      Score
----------  -------
reject        0
borderline    0.333
regular       0.667
accepted      1


In [66]:
#@title 2. Translation
import pandas as pd

arrow = {
  "up": "\u2191",
  "down": "\u2193",
  "neg": "\u00AC",
  "V": "V",
}

def get_srh(review, decision2score):
  decisions = review["decision"].replace(' ', '').split('-')

  scores = [decision2score[decision] for decision in decisions]

  srh = []
  for score in scores:
    id = chr(int(f"0x208{review['paper']}", base=16))
    if score == min(scores) and score != 0:
      _arrow = arrow['up']
      srh.append(f"{_arrow}{score}:p{id}")
    if score == max(scores) and score != 1:
      _arrow = arrow['down']
      srh.append(f"{_arrow}{score}:p{id}")

  return ','.join(srh)


def get_classical_literal(decision_srh, decision2score):
  scores = sorted([s for s in decision2score.values()])

  cs = []
  for srh in decision_srh.split(','):
    direction = srh[0]

    id = srh[-2:]
    if direction == arrow['up']:
      cs.append(f"r{srh}")
    else:
      score = float(srh.split(':')[0][1:])
      next_score = scores[scores.index(score)+1]
      cs.append(f"{arrow['neg']}r{arrow['up']}{next_score}:{id}")

  return ','.join(cs)


def process_reviews(decision2score):
  reviews = pd.read_csv("reviews.txt")
  reviews['srh'] = reviews.apply(lambda x: get_srh(x, decision2score), axis=1)
  reviews['cs'] = reviews['srh'].apply(lambda x: get_classical_literal(x, decision2score))
  return reviews

reviews = process_reviews(read_decisions())
print(tabulate(reviews, headers="keys", showindex=False))

  reviewer    paper  decision                   srh                  cs
----------  -------  -------------------------  -------------------  ----------------------
         1        1  regular                    ↑0.667:p₁,↓0.667:p₁  r↑0.667:p₁,¬r↑1.0:p₁
         1        2  reject-borderline          ↓0.333:p₂            ¬r↑0.667:p₂
         2        1  reject-borderline-regular  ↓0.667:p₁            ¬r↑1.0:p₁
         2        2  regular-accepted           ↑0.667:p₂            r↑0.667:p₂
         3        3  accepted                   ↑1.0:p₃              r↑1.0:p₃
         3        4  borderline                 ↑0.333:p₄,↓0.333:p₄  r↑0.333:p₄,¬r↑0.667:p₄
         4        3  regular-accepted           ↑0.667:p₃            r↑0.667:p₃
         4        4  borderline                 ↑0.333:p₄,↓0.333:p₄  r↑0.333:p₄,¬r↑0.667:p₄


In [84]:
#@title 3. Knowledge Base
knowledge_base = {}
for reviewer_id in reviews["reviewer"].unique():
  knowledge_base[f"K{reviewer_id}"] = reviews[reviews["reviewer"] == reviewer_id]["cs"].values.tolist()
print(tabulate(knowledge_base, headers="keys"))

K1                    K2          K3                      K4
--------------------  ----------  ----------------------  ----------------------
r↑0.667:p₁,¬r↑1.0:p₁  ¬r↑1.0:p₁   r↑1.0:p₃                r↑0.667:p₃
¬r↑0.667:p₂           r↑0.667:p₂  r↑0.333:p₄,¬r↑0.667:p₄  r↑0.333:p₄,¬r↑0.667:p₄


In [87]:
#@title 4. Encode constraints
constraints = []
paper_ids = reviews["paper"].unique()

for p1, p2 in zip(paper_ids[::2], paper_ids[1::2]):
  p_id1 = chr(int(f"0x208{p1}", base=16))
  p_id2 = chr(int(f"0x208{p2}", base=16))
  constraints.append(f"{arrow['up']}0.667:p{p_id1}{arrow['V']}{arrow['up']}0.667:p{p_id2}")

print("Cada revisor ha d'avaluar, com a mímin, un article com a `accepted` o `regular`\n")
print(tabulate({"Constraints": constraints}, headers="keys"))

tmp = ','.join(constraints).replace('V', ',')
literal = get_classical_literal(tmp, decision2score).replace(',', 'V')
print("\n")
print(f"Classical literal form: {literal}")

Cada revisor ha d'avaluar, com a mímin, un article com a `accepted` o `regular`

Constraints
-------------------
↑0.667:p₁V↑0.667:p₂
↑0.667:p₃V↑0.667:p₄


Classical literal form: r↑0.667:p₁Vr↑0.667:p₂Vr↑0.667:p₃Vr↑0.667:p₄


In [83]:
#@title 6. Generate D (whatever that might be)

import itertools as itt

def generate_D():
  N_ = [n for n in N if n > 0]
  # P =

  D = []
  for p in reviews["paper"].unique():
    D_ = []
    for j, k in itt.combinations(N_, r=2):
      p_id = chr(int(f"0x208{p}", base=16))
      d = f"{arrow['neg']}r{arrow['up']}{k}:p{p_id}{arrow['V']}r{arrow['up']}{j}:p{p_id}"
      D_.append(d)
    if p % 2 == 1:
      D.append(D_)
    else:
      D[p//2-1].extend(D_)
  return D

def print_D(D):
  for formula in D:
    for clause in formula[:-1]:
      print(f"({clause})", end=' & ')
    print(f"({formula[-1]})")

D = generate_D()
print_D(D)

(¬r↑0.667:p₁Vr↑0.333:p₁) & (¬r↑1.0:p₁Vr↑0.333:p₁) & (¬r↑1.0:p₁Vr↑0.667:p₁) & (¬r↑0.667:p₂Vr↑0.333:p₂) & (¬r↑1.0:p₂Vr↑0.333:p₂) & (¬r↑1.0:p₂Vr↑0.667:p₂)
(¬r↑0.667:p₃Vr↑0.333:p₃) & (¬r↑1.0:p₃Vr↑0.333:p₃) & (¬r↑1.0:p₃Vr↑0.667:p₃) & (¬r↑0.667:p₄Vr↑0.333:p₄) & (¬r↑1.0:p₄Vr↑0.333:p₄) & (¬r↑1.0:p₄Vr↑0.667:p₄)


In [89]:
#@title 7. Find models of K*
scores = [s for s in reversed([s for s in decision2score.values()][1:])]
paper_ids = reviews["paper"].unique()

vars = []
for p in paper_ids:
  p_id = chr(int(f"0x208{p}", base=16))
  vars_ = [
    f"r{arrow['up']}{score}:p{p_id}" for score in scores
  ]
  if p % 2 == 1:
    vars.append(vars_)
  else:
    vars[p//2-1].extend(vars_)

# si no apareix la variable, qualsevol valor.
# si apareix en positiu, 1
# si apareix en negatiu, 0

models = []
for K, K_clauses in knowledge_base.items():
  K_vars = vars[int((int(K[1])-1)/2)] # + vars[int((int(K[1]))/2)]
  models.append([])
  K_clauses_expanded = []
  for clause in K_clauses:
    K_clauses_expanded.extend(clause.split(','))

  for i, v in enumerate(K_vars):
    if v in K_clauses_expanded:
      # print(f"{v} in {K_clauses_expanded}")
      models[-1].append(1)
    elif f"{arrow['neg']}{v}" in K_clauses_expanded:
      # print(f"{arrow['neg']}{v} in {K_clauses_expanded}")
      models[-1].append(0)
    else:
      # print(f"{v} NOT in {K_clauses_expanded}")
      # print(f"{arrow['neg']}{v} NOT in {K_clauses_expanded}")
      models[-1].append('*')

print(tabulate(models))

-  -  -  -  -  -
0  1  *  *  0  *
0  *  *  *  1  *
1  *  *  *  0  1
*  1  *  *  0  1
-  -  -  -  -  -


# Step 8: Find models of D & constraint

In [25]:
# Models of D[0] and constraints[0]
# Models of D[1] and constraints[1]

print(D[0])
print(constraints[0])

['¬r↑0.667:p₁Vr↑0.333:p₁', '¬r↑1.0:p₁Vr↑0.333:p₁', '¬r↑1.0:p₁Vr↑0.667:p₁', '¬r↑0.667:p₂Vr↑0.333:p₂', '¬r↑1.0:p₂Vr↑0.333:p₂', '¬r↑1.0:p₂Vr↑0.667:p₂']
↑0.667:p₁V↑0.667:p₂


# Step 9: Find intersection of models in Steps 7 & 8